In [1]:
!pip -q install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import requests
import http,json
import openai
import os
import pandas as pd
from datetime import datetime

# import local library
import sys
# sys.path.append(r'/content/drive/Othercomputers/My Laptop/NIT6001/CMG_prog_v2')
sys.path.append(os.path.abspath('/content/drive/Othercomputers/My Laptop/NIT6001/CMG_prog_v2'))
from GPTCall import ask_chatgpt
from SaveExcel_v2 import update_sheet_preserving_format, update_list_sheets_preserving_format

In [4]:
mypath = r"/content/drive/Othercomputers/My Laptop/NIT6001/CMG_dataset"
myfile = r"/MSDManual/Cognitive Map Graph Processing v4 2024.03.14.xlsx"
process_knowledge_filename = r"/MSDManual/CMG_article_process_knowledge.xlsx"

# Use os.path.join to create the full paths
process_knowledge_file_fullpath = os.path.join(mypath, process_knowledge_filename.lstrip('/'))
myexcelfile = os.path.join(mypath, myfile.lstrip('/'))

In [5]:
import time
def keyfunction_readme():
    i=i
    # key function steps
    #-------------------------------------

    # 1.  In mypath folder, there is an excel file called Cognitive Map Graph Processing v3 2024.02.14.xlsx
    # 2.  The excel file hastwo sheets, one are paragraphs, and one are cognitive map graph sentences
        # Sheet name: paragraphs
        #Columns: ['ID', 'Paragraph text', 'url', 'category labels', 'summarised key points in simple sentences', 'processing user', 'processing date']

        #Sheet name: sentences
        #Columns: ['ID', 'paragraph ID', 'CMG Auto with GPT', 'CMG by Human Expert', 'Justification of the correction', 'processing user', 'processing date', 'correction user', 'corrction date']

    # 3. Read the original text to a dataframe called df, run through it row by row, call ChatGPT API,
    #     use the following myprompt to summarise the key points of the text:
    #     myprompt="1) Summarise the key point, or information/knowledge, of the following text,
    #               2) use simple structrued setnecnes;  3) each sentence should be self contained, avoid using propositions
    #               to refer to entities in ealrier sentences; 4) response in format of  Key Points =  'the key points' "
    # 4. Parse the ChatGPT response to extract the keypoints, and update the keypoints in col4 of the dataframe df
    # 5. For each row in col4, ask chatGPT API to convert the sentences into  head, relation, tail structure. For example,
    #        Acute kidney injury is a rapid decrease in renal function over days to weeks. will be separated into:
    #        Acute kidney injury, is, a rapid decrease in renal function (duration: over days to weeks).
    #     Here we use () to enclose properties of the head, tail or relation. Multiple properties can be separated with comma.
    # 6. Note that a sentence may not have a tail, which can be represented with a -. For example,
    #       Acute kidney injury can be fatal.   can be converted as
    #       Acute kidney injury, can be fatal, -.
    # 7. For a sentence with a sub clause, use [] to enclose the main sentence and the sub clause. Use []-(connecting word)-[]. for the converted sentence.
    #      for example,  Tom had AKI when he was 50.  will be converted as
    #                   [Tom, had AKI, -]-(when)-[Tom, was 50, -]
    #      note the relationship needs to be meaningful. is, have, get are too short to represent the meaning of the relation.
    # 8. Resonse will be in format of  FCM scripts= ' ****'
    # 9. Extract FCM scripts from the response, and write to col5 of df

def main():
    print ("main function started \n--------------------")
    time_started=time.time()

    # myexcelfile=mypath+'Cognitive Map Graph Processing v4 2024.03.14.xlsx'
    #check_excelfile_info(myexcelfile)

    df_sentences = pd.read_excel(myexcelfile, sheet_name='sentences')
    row_start=0;       row_end=0 # end is 0 means to the end
    convertsentence_toCMG(df_sentences,row_start, row_end )

    update_sheet_preserving_format(myexcelfile, 'sentences', df_sentences)

    time_finished=time.time()
    timeused=time_finished-time_started
    print("Time used=", round(timeused,2))


def check_excelfile_info(myexcelfile):
# check the sheet names and columns in the excel file
     # Iterate through all sheets
    print(myexcelfile)
    xls = pd.ExcelFile(myexcelfile)

    for sheet_name in xls.sheet_names:
        # Read each sheet
        df = pd.read_excel(xls, sheet_name)

        # Print the sheet name and its columns
        print(f"Sheet name: {sheet_name}")
        print("Columns:", df.columns.tolist())



def convertCMG_prompt():
    # changed to read prompts from excel file

    df = pd.read_excel(process_knowledge_file_fullpath, sheet_name="knowledge", engine='openpyxl')
    filtered_df = df[(df['knowledge_area'] == "step3_convert_sentence_to_cognitive_map_graph") ]
    if filtered_df.empty:
        myprompt="Could not read the knowledge on step3_convert_sentence_to_cognitive_map_graph.\n"
    else:
        myprompt = '\n'.join(filtered_df['knowledge'].astype(str))
    return myprompt +"\n Here is the sentence:"


def convertsentence_toCMG(df_sentences,row_start, row_end ):
    print ("convertsentence_toCMG function started \n--------------------")
    myprompt=convertCMG_prompt()
    if row_end == 0:   row_end = df_sentences.index[-1]

    for index in range(row_start, row_end + 1):  # +1 because the range end is exclusive
        if index < len(df_sentences) : # Check to ensure index is within DataFrame bounds
            if df_sentences.at[index, 'processed'] !='Yes':
                sentence_text = df_sentences.at[index, 'Sentence text']
                response_text = ask_chatgpt(myprompt, sentence_text)
                df_sentences.at[index, 'CMG Auto with GPT'] = response_text
                df_sentences.at[index, 'processed'] = 'Yes'

        else:
            break

    return df_sentences

In [6]:
main()

main function started 
--------------------
convertsentence_toCMG function started 
--------------------
Time used= 13.4


In [15]:
# v1
import openpyxl
import pandas as pd

# Load the Excel file
file_path = myexcelfile
sheet_name = "sentences"

# Read the specified columns from the sheet
df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=["CMG Auto with GPT", "Paragraph ID"])

# Create a list to hold the new rows for the new sheet
new_rows = []

# Iterate through each row in the dataframe
for index, row in df.iterrows():
    cmg_text = row["CMG Auto with GPT"]
    paragraph_id = row["Paragraph ID"]

    # Split the text by empty lines and remove empty entries
    sentences = [sentence.strip() for sentence in cmg_text.split('\n\n') if sentence.strip()]

    # Create new rows for each split sentence
    for sentence_id, sentence in enumerate(sentences, start=1):
        new_rows.append({
            "CMG Auto with GPT single line": sentence,
            "Paragraph ID": paragraph_id,
            "Sentence ID": sentence_id
        })

# Create a new dataframe from the new rows
new_df = pd.DataFrame(new_rows)

# Write the new dataframe to the "sentences2" sheet in the same Excel file
with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    new_df.to_excel(writer, sheet_name="sentences2", index=False)

print("Processing complete.")


Processing complete.


In [21]:
# v2
import openpyxl
import pandas as pd

# Load the Excel file
file_path = myexcelfile
sheet_name = "sentences"

# Read the specified columns from the sheet
df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=["CMG Auto with GPT", "Paragraph ID"])

# Check if the "sentences2" sheet exists
book = openpyxl.load_workbook(file_path)
if "sentences2" in book.sheetnames:
    # Load the existing "sentences2" sheet
    existing_df = pd.read_excel(file_path, sheet_name="sentences2")
else:
    # Create an empty DataFrame if the sheet doesn't exist
    existing_df = pd.DataFrame(columns=[
        "CMG Auto with GPT single line",
        "Paragraph ID",
        "Sentence ID",
        "Processed",
        "Justification of the correction",
        "CMG by Human Expert"
    ])

# Create a list to hold the new rows for the new sheet
new_rows = []

# Iterate through each row in the dataframe
for index, row in df.iterrows():
    cmg_text = row["CMG Auto with GPT"]
    paragraph_id = row["Paragraph ID"]

    # Split the text by empty lines and remove empty entries
    sentences = [sentence.strip() for sentence in cmg_text.split('\n\n') if sentence.strip()]

    # Create new rows for each split sentence
    for sentence_id, sentence in enumerate(sentences, start=1):
        new_row = {
            "CMG Auto with GPT single line": sentence,
            "Paragraph ID": paragraph_id,
            "Sentence ID": sentence_id,
            "Processed": "No",  # Set default processed value to "No"
            "Justification of the correction": "",  # Default empty value
            "CMG by Human Expert": ""  # Default empty value
        }

        # Check if the existing row has the same Paragraph ID and Sentence ID and if it is marked as processed
        if not existing_df.empty:
            existing_row = existing_df[(existing_df["Paragraph ID"] == paragraph_id) & (existing_df["Sentence ID"] == sentence_id)]
            if not existing_row.empty and existing_row["Processed"].values[0] == "Yes":
                # If the existing row is marked as processed, retain the existing row's values
                new_row["Processed"] = "Yes"
                new_row["Justification of the correction"] = existing_row["Justification of the correction"].values[0]
                new_row["CMG by Human Expert"] = existing_row["CMG by Human Expert"].values[0]
            elif not existing_row.empty:
                # If the existing row is not marked as processed, retain existing "Justification of the correction" and "CMG by Human Expert" values
                new_row["Justification of the correction"] = existing_row["Justification of the correction"].values[0]
                new_row["CMG by Human Expert"] = existing_row["CMG by Human Expert"].values[0]

        new_rows.append(new_row)

# Create a new dataframe from the new rows
new_df = pd.DataFrame(new_rows)

# Combine with existing dataframe to ensure we retain all "Processed" rows without duplication
if not existing_df.empty:
    updated_df = pd.concat([
        existing_df[~existing_df["Paragraph ID"].isin(new_df["Paragraph ID"])],
        new_df
    ], ignore_index=True)
else:
    updated_df = new_df

# Write the updated dataframe to the "sentences2" sheet in the same Excel file
with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    updated_df.to_excel(writer, sheet_name="sentences2", index=False)

print("Processing complete.")

Processing complete.


In [7]:
# v3
import openpyxl
import pandas as pd

# Load the Excel file
file_path = myexcelfile
sheet_name = "sentences"

# Read the specified columns from the sheet
df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=["CMG Auto with GPT", "Paragraph ID"])

# Check if the "sentences2" sheet exists
book = openpyxl.load_workbook(file_path)
if "sentences2" in book.sheetnames:
    # Load the existing "sentences2" sheet
    existing_df = pd.read_excel(file_path, sheet_name="sentences2")
else:
    # Create an empty DataFrame if the sheet doesn't exist
    existing_df = pd.DataFrame(columns=[
        "CMG Auto with GPT single line",
        "Paragraph ID",
        "Sentence ID",
        "Processed",
        "Justification of the correction",
        "CMG by Human Expert"
    ])

# Create a list to hold the new rows for the new sheet
new_rows = []

# Iterate through each row in the dataframe
for index, row in df.iterrows():
    cmg_text = row["CMG Auto with GPT"]
    paragraph_id = row["Paragraph ID"]

    # Split the text by empty lines and remove empty entries
    sentences = [sentence.strip() for sentence in cmg_text.split('\n\n') if sentence.strip()]

    # Create new rows for each split sentence
    for sentence_id, sentence in enumerate(sentences, start=1):
        new_row = {
            "CMG Auto with GPT single line": sentence,
            "Paragraph ID": paragraph_id,
            "Sentence ID": sentence_id,
            "Processed": "No",  # Set default processed value to "No"
            "Justification of the correction": "",  # Default empty value
            "CMG by Human Expert": ""  # Default empty value
        }

        # Check if the existing row has the same Paragraph ID and Sentence ID and if it is marked as processed
        if not existing_df.empty:
            existing_row = existing_df[(existing_df["Paragraph ID"] == paragraph_id) & (existing_df["Sentence ID"] == sentence_id)]
            if not existing_row.empty and existing_row["Processed"].values[0] == "Yes":
                # If the existing row is marked as processed, retain the existing row's values
                new_row["Processed"] = "Yes"
                new_row["Justification of the correction"] = existing_row["Justification of the correction"].values[0]
                new_row["CMG by Human Expert"] = existing_row["CMG by Human Expert"].values[0]

        new_rows.append(new_row)

# Create a new dataframe from the new rows
new_df = pd.DataFrame(new_rows)

# Combine with existing dataframe to ensure we retain all "Processed" rows without duplication
if not existing_df.empty:
    updated_df = pd.concat([
        existing_df[~existing_df["Paragraph ID"].isin(new_df["Paragraph ID"])],
        new_df
    ], ignore_index=True)
else:
    updated_df = new_df

# Ensure "Justification of the correction" and "CMG by Human Expert" are emptied when "Processed" is "No"
updated_df.loc[updated_df["Processed"] == "No", ["Justification of the correction", "CMG by Human Expert"]] = ""

# Write the updated dataframe to the "sentences2" sheet in the same Excel file
with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    updated_df.to_excel(writer, sheet_name="sentences2", index=False)

print("Processing complete.")


Processing complete.


In [ ]:
#!pip install gspread pandas gspread-dataframe
import gspread
import pandas as pd
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2.service_account import Credentials

# Path to your service account key file
creds_file = 'path/to/your/service-account-file.json'

# Define the scope
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

# Authenticate using the service account
creds = Credentials.from_service_account_file(creds_file, scopes=scope)
client = gspread.authorize(creds)

# Open the Google Sheets file by name
spreadsheet = client.open("Cognitive Map Graph Processing v4 2024.03.14")

# Load the specific sheet into a DataFrame
sheet_name = "sentences"
sheet = spreadsheet.worksheet(sheet_name)
df = get_as_dataframe(sheet, evaluate_formulas=True)

# Drop any completely empty rows that might have been read in
df.dropna(how='all', inplace=True)

# Create a list to hold the new rows for the new sheet
new_rows = []

# Iterate through each row in the dataframe
for index, row in df.iterrows():
    cmg_text = row["CMG Auto with GPT"]
    paragraph_id = row["Paragraph ID"]

    # Split the text by empty lines and remove empty entries
    sentences = [sentence.strip() for sentence in cmg_text.split('\n\n') if sentence.strip()]

    # Create new rows for each split sentence
    for sentence_id, sentence in enumerate(sentences, start=1):
        new_rows.append({
            "CMG Auto with GPT single line": sentence,
            "Paragraph ID": paragraph_id,
            "Sentence ID": sentence_id
        })

# Create a new dataframe from the new rows
new_df = pd.DataFrame(new_rows)

# Write the new dataframe to a new sheet "sentences2"
new_sheet_name = "sentences2"
try:
    spreadsheet.del_worksheet(spreadsheet.worksheet(new_sheet_name))
except gspread.exceptions.WorksheetNotFound:
    pass
new_sheet = spreadsheet.add_worksheet(title=new_sheet_name, rows=new_df.shape[0], cols=new_df.shape[1])
set_with_dataframe(new_sheet, new_df)

print("Processing complete.")
